In [51]:
# import all necessary dependencies
import pandas as pd
import psycopg2
import sklearn as skl
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from sklearn.preprocessing import StandardScaler
import numpy as np

In [52]:
# read in the raw data files
season_2018_df = pd.read_excel('wr_season_2018.xlsx')
season_2019_df = pd.read_excel('wr_season_2019.xlsx')
season_2020_df = pd.read_excel('wr_season_2020.xlsx')
season_2021_df = pd.read_excel('wr_season_2021.xlsx')
season_2022_df = pd.read_excel('wr_season_2022.xlsx')

## Data Cleaning

In [53]:
# preview new dataframe
season_2018_df.head()

,Rank,Name,Team,Pos,GMS,TGTS,REC,PCT,YDS,TD,...,Y/T,Y/R,ATT,YDS.1,AVG,TD.1,FUM,LST,FPTS/G,FPTS
0,1,Tyreek Hill,KC,WR,16,137,87,63.5,1479,12,...,10.8,17.0,22,151,6.9,1,0,0,15.1,241.0
1,2,Antonio Brown,PIT,WR,15,168,104,61.9,1297,15,...,7.7,12.5,0,0,0.0,0,0,0,14.6,219.7
2,3,Davante Adams,GB,WR,15,169,111,65.7,1386,13,...,8.2,12.5,0,0,0.0,0,0,0,14.6,218.6
3,4,DeAndre Hopkins,HOU,WR,16,163,115,70.6,1572,11,...,9.6,13.7,1,-7,-7.0,0,2,2,13.7,218.5
4,5,Julio Jones,ATL,WR,16,170,113,66.5,1677,8,...,9.9,14.8,2,12,6.0,0,2,2,13.3,212.9


In [54]:
# drop rank column from all dataframes since we don't need this value for the model
season_2018_df = season_2018_df.drop(columns=['Rank'])
season_2019_df = season_2019_df.drop(columns=['Rank'])
season_2020_df = season_2020_df.drop(columns=['Rank'])
season_2021_df = season_2021_df.drop(columns=['Rank'])
season_2022_df = season_2022_df.drop(columns=['Rank'])

In [55]:
# rename columns to be clear terminology
season_2018_df = season_2018_df.rename(columns={'Pos': 'Position', 'GMS': 'Games', 'REC': 'Receptions', 'TGTS': 'Targets', 'PCT': 'Percentage',
                               'YDS': 'Yards', 'TD': 'Touchdowns', 'LNG': 'Long', 'Y/T': 'Yards_per_target', 'Y/R': 'Yards_per_reception', 'ATT': 'Attempts',
                               'TD.1': 'Rushing_touchdown', 'YDS.1': 'Rushing_yards', 'AVG': 'Average_rushing_yards', 'FUM': 'Fumbles', 'LST': 'Lost_yards',
                               'FPTS/G': 'Fantasy_points_per_game', 'FPTS': 'Fantasy_points'})

In [56]:
# rename columns to be clear terminology
season_2019_df = season_2019_df.rename(columns={'Pos': 'Position', 'GMS': 'Games', 'REC': 'Receptions', 'TGTS': 'Targets', 'PCT': 'Percentage',
                               'YDS': 'Yards', 'TD': 'Touchdowns', 'LNG': 'Long', 'Y/T': 'Yards_per_target', 'Y/R': 'Yards_per_reception', 'ATT': 'Attempts',
                               'TD.1': 'Rushing_touchdown', 'YDS.1': 'Rushing_yards', 'AVG': 'Average_rushing_yards', 'FUM': 'Fumbles', 'LST': 'Lost_yards',
                               'FPTS/G': 'Fantasy_points_per_game', 'FPTS': 'Fantasy_points'})

In [57]:
# rename columns to be clear terminology
season_2020_df = season_2020_df.rename(columns={'Pos': 'Position', 'GMS': 'Games', 'TGTS': 'Targets', 'REC': 'Receptions', 'PCT': 'Percentage',
                               'YDS': 'Yards', 'TD': 'Touchdowns', 'LNG': 'Long', 'Y/T': 'Yards_per_target', 'Y/R': 'Yards_per_reception', 'ATT': 'Attempts',
                               'TD.1': 'Rushing_touchdown', 'YDS.1': 'Rushing_yards', 'AVG': 'Average_rushing_yards', 'FUM': 'Fumbles', 'LST': 'Lost_yards',
                               'FPTS/G': 'Fantasy_points_per_game', 'FPTS': 'Fantasy_points'})

In [58]:
# rename columns to be clear terminology
season_2021_df = season_2021_df.rename(columns={'Pos': 'Position', 'GMS': 'Games', 'TGTS': 'Targets', 'REC': 'Receptions', 'PCT': 'Percentage',
                               'YDS': 'Yards', 'TD': 'Touchdowns', 'LNG': 'Long', 'Y/T': 'Yards_per_target', 'Y/R': 'Yards_per_reception', 'ATT': 'Attempts',
                               'TD.1': 'Rushing_touchdown', 'YDS.1': 'Rushing_yards', 'AVG': 'Average_rushing_yards', 'FUM': 'Fumbles', 'LST': 'Lost_yards',
                               'FPTS/G': 'Fantasy_points_per_game', 'FPTS': 'Fantasy_points'})

In [59]:
# rename columns to be clear terminology
season_2022_df = season_2022_df.rename(columns={'REC': 'Receptions', 'TGTS': 'Targets', 'PCT': 'Percentage',
                               'YDS': 'Yards', 'TD': 'Touchdowns', 'LNG': 'Long', 'Y/T': 'Yards_per_target', 'Y/R': 'Yards_per_reception', 'ATT': 'Attempts',
                               'TD.1': 'Rushing_touchdown', 'YDS.1': 'Rushing_yards', 'AVG': 'Average_rushing_yards', 'FUM': 'Fumbles', 'LST': 'Lost_yards',
                               'FPTS/G': 'Fantasy_points_per_game', 'FPTS': 'Fantasy_points'})

In [60]:
# check to make sure columns have been renamed
season_2018_df.head()

,Name,Team,Position,Games,Targets,Receptions,Percentage,Yards,Touchdowns,Long,Yards_per_target,Yards_per_reception,Attempts,Rushing_yards,Average_rushing_yards,Rushing_touchdown,Fumbles,Lost_yards,Fantasy_points_per_game,Fantasy_points
0,Tyreek Hill,KC,WR,16,137,87,63.5,1479,12,75,10.8,17.0,22,151,6.9,1,0,0,15.1,241.0
1,Antonio Brown,PIT,WR,15,168,104,61.9,1297,15,78,7.7,12.5,0,0,0.0,0,0,0,14.6,219.7
2,Davante Adams,GB,WR,15,169,111,65.7,1386,13,57,8.2,12.5,0,0,0.0,0,0,0,14.6,218.6
3,DeAndre Hopkins,HOU,WR,16,163,115,70.6,1572,11,49,9.6,13.7,1,-7,-7.0,0,2,2,13.7,218.5
4,Julio Jones,ATL,WR,16,170,113,66.5,1677,8,58,9.9,14.8,2,12,6.0,0,2,2,13.3,212.9


In [61]:
# add column to each dataframe that contains the year of play
season_2018_df['Year'] = '2018'
season_2019_df['Year'] = '2019'
season_2020_df['Year'] = '2020'
season_2021_df['Year'] = '2021'

In [62]:
# concat all dataframes that will be used to train the model
all_dfs = [season_2018_df, season_2019_df, season_2020_df, season_2021_df]
results = pd.concat(all_dfs)

In [63]:
# write the clean dataframes to csv for future use
season_2018_df.to_csv('wr_season_2018_clean.csv', index=False)

In [64]:
# write the clean dataframes to csv for future use
season_2019_df.to_csv('wr_season_2019_clean.csv', index=False)

In [65]:
# write the clean dataframes to csv for future use
season_2020_df.to_csv('wr_season_2020_clean.csv', index=False)

In [66]:
# write the clean dataframes to csv for future use
season_2021_df.to_csv('wr_season_2021_clean.csv', index=False)

In [67]:
# write the clean dataframes to csv for future use
season_2022_df.to_csv('wr_season_2022_clean.csv', index=False)

## Data Preprocessing

In [68]:
# drop unnecessary columns before fitting model
results = results.drop(columns=['Team', 'Position', 'Year', 'Games'])

In [69]:
# set name column as index
results = results.set_index('Name')

In [70]:
# view new df containing converted data
results

,Targets,Receptions,Percentage,Yards,Touchdowns,Long,Yards_per_target,Yards_per_reception,Attempts,Rushing_yards,Average_rushing_yards,Rushing_touchdown,Fumbles,Lost_yards,Fantasy_points_per_game,Fantasy_points
Name,,,,,,,,,,,,,,,,
Tyreek Hill,137,87,63.5,1479,12,75,10.8,17.0,22,151,6.9,1,0,0,15.1,241.0
Antonio Brown,168,104,61.9,1297,15,78,7.7,12.5,0,0,0.0,0,0,0,14.6,219.7
Davante Adams,169,111,65.7,1386,13,57,8.2,12.5,0,0,0.0,0,0,0,14.6,218.6
DeAndre Hopkins,163,115,70.6,1572,11,49,9.6,13.7,1,-7,-7.0,0,2,2,13.7,218.5
Julio Jones,170,113,66.5,1677,8,58,9.9,14.8,2,12,6.0,0,2,2,13.3,212.9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Malik Taylor,3,2,66.7,14,0,7,4.7,7.0,0,0,0.0,0,1,1,-0.1,-0.6
Racey McMath,6,2,33.3,8,0,9,1.3,4.0,0,0,0.0,0,1,1,-0.1,-1.2
Travis Benjamin,5,0,0.0,0,0,0,0.0,0.0,0,0,0.0,0,1,1,-0.2,-2.0


In [71]:
# y is the target and x is the features
y = results['Touchdowns']
X = results.drop(columns=['Touchdowns'])

# create the training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1, test_size=.20)

In [72]:
# create a StandardScaler instance
scaler = StandardScaler()

# fit the StandardScaler
X_scaler = scaler.fit(X_train)

# scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [73]:
# check the shape of X_test dataset for creating the model
X_test.shape

(198, 15)

## Model Creation

In [74]:
# define the model
nn = Sequential()

# first hidden layer
nn.add(Dense(units=70, activation = 'relu', input_dim = X_test.shape[1]))

# second hidden layer
nn.add(Dense(units=40, activation='relu'))

# third hidden layer
nn.add(Dense(units=20, activation='relu'))

# output layer
nn.add(Dense(units=1, activation='relu'))

# check the structure of the model
nn.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_4 (Dense)             (None, 70)                1120      
                                                                 
 dense_5 (Dense)             (None, 40)                2840      
                                                                 
 dense_6 (Dense)             (None, 20)                820       
                                                                 
 dense_7 (Dense)             (None, 1)                 21        
                                                                 
Total params: 4801 (18.75 KB)
Trainable params: 4801 (18.75 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [75]:
# compile the model (mae or mse loss functions)
nn.compile(loss="mse", optimizer="adam", metrics=["mae"])

In [76]:
# train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=60)

Epoch 1/60
25/25 [==============================] - 0s 604us/step - loss: 11.3822 - mae: 2.0405
Epoch 2/60
25/25 [==============================] - 0s 567us/step - loss: 8.2257 - mae: 1.7345
Epoch 3/60
25/25 [==============================] - 0s 539us/step - loss: 2.7995 - mae: 1.0374
Epoch 4/60
25/25 [==============================] - 0s 535us/step - loss: 1.7359 - mae: 0.8190
Epoch 5/60
25/25 [==============================] - 0s 584us/step - loss: 1.4476 - mae: 0.7548
Epoch 6/60
25/25 [==============================] - 0s 586us/step - loss: 1.3588 - mae: 0.7397
Epoch 7/60
25/25 [==============================] - 0s 535us/step - loss: 1.2627 - mae: 0.7148
Epoch 8/60
25/25 [==============================] - 0s 591us/step - loss: 1.1898 - mae: 0.6961
Epoch 9/60
25/25 [==============================] - 0s 597us/step - loss: 1.0917 - mae: 0.6724
Epoch 10/60
25/25 [==============================] - 0s 601us/step - loss: 1.0138 - mae: 0.6365
Epoch 11/60
25/25 [=============================

In [77]:
# evaluate the model using the test data
model_loss, mae = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Mean Absolute Error: {mae}")

7/7 - 0s - loss: 0.1243 - mae: 0.1900 - 39ms/epoch - 6ms/step
Loss: 0.12431354075670242, Mean Absolute Error: 0.18997028470039368


## Predictions

In [78]:
# set name column as index
season_2022_df = season_2022_df.set_index('Name')

In [79]:
# drop touchdown column so this is not factored into predictions
season_2022_df = season_2022_df.drop(columns='Touchdowns')

In [80]:
# display dataframe to verify the structure is correct
season_2022_df.head()

,Targets,Receptions,Percentage,Yards,Long,Yards_per_target,Yards_per_reception,Attempts,Rushing_yards,Average_rushing_yards,Rushing_touchdown,Fumbles,Lost_yards,Fantasy_points_per_game,Fantasy_points
Name,,,,,,,,,,,,,,,
Justin Jefferson,184,128,69.6,1809,64,9.8,14.1,4,24,6.0,1,0,0,14.2,240.66
Davante Adams,180,100,55.6,1516,60,8.4,15.2,3,-1,-0.3,0,1,0,13.9,235.50
Tyreek Hill,170,119,70.0,1710,64,10.1,14.4,7,32,4.6,2,1,0,13.4,228.20
A.J. Brown,146,88,60.3,1496,78,10.2,17.0,0,0,0.0,0,2,2,12.4,211.60
Stefon Diggs,154,108,70.1,1429,53,9.3,13.2,1,-3,-3.0,0,1,0,13.0,208.60


In [81]:
# create variable to input any given player for making predictions
selected_player = season_2022_df.loc['Courtland Sutton']

# view selected player stats
selected_player

Targets                    109.0
Receptions                  64.0
Percentage                  58.7
Yards                      829.0
Long                        51.0
Yards_per_target             7.6
Yards_per_reception         13.0
Attempts                     1.0
Rushing_yards                5.0
Average_rushing_yards        5.0
Rushing_touchdown            0.0
Fumbles                      0.0
Lost_yards                   0.0
Fantasy_points_per_game      6.4
Fantasy_points              95.4
Name: Courtland Sutton, dtype: float64

In [82]:
# convert player data to array that contains the values of features we want to provide the model
# this is how it will make its predictions
player_array = np.array(selected_player)

# view structure of the array
player_array

array([109. ,  64. ,  58.7, 829. ,  51. ,   7.6,  13. ,   1. ,   5. ,
         5. ,   0. ,   0. ,   0. ,   6.4,  95.4])

In [83]:
# convert the player array to a shape usable by the X_scaler
converted_array = player_array.reshape(1, -1)

In [84]:
# scale the data using same X_scaler as used for the model
record_to_predict = X_scaler.transform(converted_array)

/Users/owner/anaconda3/envs/dev/lib/python3.11/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


In [85]:
# give the model player feature data for predictions
y_pred = nn.predict(record_to_predict)

1/1 [==============================] - 0s 21ms/step


In [86]:
# view the predictions made by the model
print(y_pred)

[[1.8081461]]


In [87]:
# create variable as array to input all of season 2022 data to make predictions with
whole_season_array = np.array(season_2022_df)

# view structure of the array
whole_season_array

array([[ 1.8400e+02,  1.2800e+02,  6.9600e+01, ...,  0.0000e+00,
         1.4200e+01,  2.4066e+02],
       [ 1.8000e+02,  1.0000e+02,  5.5600e+01, ...,  0.0000e+00,
         1.3900e+01,  2.3550e+02],
       [ 1.7000e+02,  1.1900e+02,  7.0000e+01, ...,  0.0000e+00,
         1.3400e+01,  2.2820e+02],
       ...,
       [ 2.0000e+00,  2.0000e+00,  1.0000e+02, ...,  1.0000e+00,
        -2.0000e-01, -7.0000e-01],
       [ 2.0000e+00,  1.0000e+00,  5.0000e+01, ...,  2.0000e+00,
        -1.0000e-01, -1.4000e+00],
       [ 0.0000e+00,  0.0000e+00,  0.0000e+00, ...,  1.0000e+00,
        -7.0000e-01, -2.0000e+00]])

In [88]:
# scale the data using same X_scaler as used for the model
season_to_predict = X_scaler.transform(whole_season_array)

/Users/owner/anaconda3/envs/dev/lib/python3.11/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


In [89]:
# give the model the season data to use for predictions
y_pred_whole_season = nn.predict(season_to_predict)

# view the predictions made by the model
print(y_pred_whole_season)

8/8 [==============================] - 0s 440us/step
[[ 8.301369  ]
 [14.373172  ]
 [ 7.528002  ]
 [11.248     ]
 [10.987959  ]
 [ 8.290208  ]
 [ 7.7849855 ]
 [ 8.484814  ]
 [ 5.22115   ]
 [ 6.3949156 ]
 [ 7.1548963 ]
 [ 8.498097  ]
 [ 8.486289  ]
 [ 4.7676077 ]
 [ 6.983316  ]
 [ 5.758488  ]
 [ 7.001092  ]
 [ 5.7829266 ]
 [ 6.4598765 ]
 [ 6.552776  ]
 [ 3.830078  ]
 [ 5.6181755 ]
 [ 4.9281745 ]
 [ 6.9750576 ]
 [ 6.8186026 ]
 [ 2.9596121 ]
 [ 3.0336719 ]
 [ 4.8151584 ]
 [ 5.7885227 ]
 [ 3.983624  ]
 [ 3.6521974 ]
 [ 6.4279246 ]
 [ 1.9844408 ]
 [ 1.2948344 ]
 [ 5.955394  ]
 [ 5.065682  ]
 [ 3.8088486 ]
 [ 4.939722  ]
 [ 2.5750575 ]
 [ 3.4317937 ]
 [ 3.221068  ]
 [ 3.858856  ]
 [ 7.5035105 ]
 [ 4.34213   ]
 [ 1.8081461 ]
 [ 0.        ]
 [ 2.93712   ]
 [ 2.7221966 ]
 [ 3.066308  ]
 [ 3.0224853 ]
 [ 1.9488493 ]
 [ 1.6312075 ]
 [ 3.5989168 ]
 [ 3.325005  ]
 [ 2.6759846 ]
 [ 4.5646496 ]
 [ 2.6285582 ]
 [ 3.0067728 ]
 [ 2.8801553 ]
 [ 1.5677017 ]
 [ 2.873739  ]
 [ 3.1027787 ]
 [ 3.1920068 ]
 [

In [90]:
# turn array of predictions into new dataframe
season_2022_predictions = pd.DataFrame({'Predicted_touchdowns': y_pred_whole_season[:, 0]})

In [91]:
# create a new column of player names by taking the index from original dataframe
season_2022_predictions['Name'] = season_2022_df.index

In [92]:
# view new predictions dataframe
season_2022_predictions

,Predicted_touchdowns,Name
0,8.301369,Justin Jefferson
1,14.373172,Davante Adams
2,7.528002,Tyreek Hill
3,11.248000,A.J. Brown
4,10.987959,Stefon Diggs
...,...,...
249,0.000000,Braylon Sanders
250,0.000000,Ihmir Smith-Marsette
251,0.000000,Deonte Harty
252,0.160218,KaVontae Turpin


In [93]:
# write dataframe to csv for future use
season_2022_predictions.to_csv('season_2022_predictions.csv')